In [32]:
import json
import pandas as pd
from datasets import Dataset
import os

file_path = "SAT_raw_predictions_gemini_2.5_flash_0.csv"

df = pd.read_csv(file_path)

SAT_DATASET_FOLDER = "/ocean/projects/cis250208p/shared/datasets/SAT"
data = []

with open(os.path.join(SAT_DATASET_FOLDER, "SAT_labeled.jsonl"), "r") as f:
    for line in f:
        data.append(json.loads(line))

ds = Dataset.from_list(data)

ds = ds.select(range(100))

In [33]:
match_list = []

for i in range(len(df)):
    pred = df.iloc[i]["prediction"]
    if not isinstance(pred, str) or len(pred.strip()) == 0:
        match_list.append(False)
        continue

    pred_index = ord(pred) - ord('A')
    if pred_index < 0 or pred_index >= len(ds[i]["answer_choices"]):
        match_list.append(False)
        continue
    answer = ds[i]["answer_choices"][pred_index]
    if answer == ds[i]["correct_answer"]:
        match_list.append(True)
    else:
        match_list.append(False)

print(match_list)

[False, True, False, False, True, True, False, True, True, True, True, True, False, True, True, False, False, True, True, True, True, True, False, True, True, True, True, True, True, True, False, False, False, True, True, False, False, True, False, True, True, False, True, False, True, True, False, True, True, False, True, True, True, True, True, True, False, False, True, True, False, False, False, True, True, False, True, False, True, False, True, True, True, False, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, False, True, False, True, False, False, True, False, True, True, False]


In [34]:
# count the number of matches
matched = sum(match_list)
print(f"Matched {matched} out of {len(df)}")

# count the number of matches for each question
question_matches = {}


Matched 66 out of 100


In [3]:
print(df.head())

   index             category  \
0      0  DISTANCE_COMPARISON   
1      1    RELATIVE_LOCATION   
2      2    RELATIVE_LOCATION   
3      3    RELATIVE_LOCATION   
4      4    RELATIVE_LOCATION   

                                            question  \
0  Which point is closer to the camera taking thi...   
1  Considering the relative positions, is iron ch...   
2  Considering the relative positions, where is b...   
3  Considering the relative positions, is extenda...   
4  Considering the relative positions, where is s...   

                                           reasoning prediction answer  \
0  The image shows a perspective view where objec...          B      C   
1  The image shows a perspective view of a 3D sce...          A  below   
2  The image shows a single 3D object that appear...        NaN  right   
3  The image depicts a 3D scene with a vanishing ...          B  below   
4  The image shows a colorful landscape painting ...          A  right   

   is_correct      

In [1]:
import json
import os

SAT_TRAIN_GENERATED_FOLDER = "/ocean/projects/cis250208p/shared/datasets/SAT/train_generated"

In [15]:
global image_idx_counter
image_idx_counter = 0

from PIL import Image
from io import BytesIO
import base64

def base64_to_image(base64_str: str) -> Image.Image:
    """Convert a base64 string back to a PIL Image."""
    img_data = base64.b64decode(base64_str)
    return Image.open(BytesIO(img_data))

def deserialize_messages(serialized):
    """Convert base64-encoded images in messages back to PIL Images."""
    global image_idx_counter
    deserialized = []
    for msg in serialized:
        new_msg = {"role": msg["role"], "content": []}
        for i, item in enumerate(msg["content"]):
            if item["type"] == "image" and "data" in item:
                # Reconstruct PIL image from base64 data
                image = base64_to_image(item["data"])
                image_path = f"{SAT_TRAIN_GENERATED_FOLDER}/images/image_{image_idx_counter}.png"
                image.save(image_path)
                image_idx_counter += 1
                new_msg["content"].append({
                    "type": "image",
                    "image": image_path
                })
                msg["content"][i + 1]["text"] = "<image>" + msg["content"][i + 1]["text"]
            else:
                new_msg["content"].append(item)
        deserialized.append(new_msg)
    return deserialized

In [16]:
data = []
with open(os.path.join(SAT_TRAIN_GENERATED_FOLDER, "conversation_1.jsonl"), "r") as f:
    for line in f:
        data.append(json.loads(line))

deserialized = deserialize_messages(data[0]["messages"])
print(deserialized)
# deserialized[0]["content"][0]["image"].save("test.png")

[{'role': 'user', 'content': [{'type': 'image', 'image': '/ocean/projects/cis250208p/shared/datasets/SAT/train_generated/images/image_0.png'}, {'type': 'text', 'text': "<image>\n### Situation Description\nGiven an image and a spatial reasoning question, we need to all entities that are included in the question.\n\n# Example 1\n[Question] You are standing at the airplane's position, facing where it is facing. Is the the person on your left or right?\n[Detect] [airplane, person]\n\n# Examples 2\n[Question] From the old man's perspective, is the person wearing a hat on the left of the green car?\n[Detect] [old man, person wearing a hat, green car]\n\n# Examples 3\n[Question] From the car's perspective, which is on the right side: the person or the tree?\n[Detect] [car, person, tree]\n\n### Your Task\nNow, given the question below, please identify the entities that are included in the question.\n\n[Question] From the camera's point of view, Considering the relative positions, is iron cheir

In [ ]:
def parse_image_data(entry):
    processed_entry = {}
    deserialized = deserialize_messages(entry["messages"])
    processed_entry["messages"] = deserialized
    processed_entry["response"] = entry["response"]
    processed_entry["conv_type"] = entry["conv_type"]
    return processed_entry


In [47]:
from datasets import Dataset
from tqdm import tqdm
import json
import os

SAT_DATASET_FOLDER = "/ocean/projects/cis250208p/shared/datasets/SAT"
data = []

with open(os.path.join(SAT_DATASET_FOLDER, "SAT_labeled.jsonl"), "r") as f:
    for line in f:
        data.append(json.loads(line))

ds = Dataset.from_list(data)

SAT_TRAIN_GENERATED_FOLDER = "/ocean/projects/cis250208p/shared/datasets/SAT/train_generated_1"

correct_idx = []
total_count = 0

image_idx_counter = 329

processed_data = []

files = os.listdir(SAT_TRAIN_GENERATED_FOLDER)
files = [f for f in files if f.endswith(".jsonl")]

total_count = len(files)
for filename in tqdm(files):
    idx = int(filename.split("_")[-1].split(".")[0])
    file_path = os.path.join(SAT_TRAIN_GENERATED_FOLDER, filename)
    with open(file_path, "r") as f:
        data = []
        for line in f:
            data.append(json.loads(line))

        try:
            answer_extracted = data[-1]["response"].split("[Answer]")[1].strip()
            answer_extracted = answer_extracted.split(".")[0].strip()
        except:
            continue

    
        try:
            pred_index = ord(answer_extracted) - ord('A')
        except:
            print(answer_extracted)
            continue

        if pred_index < 0 or pred_index >= len(ds[idx]["answer_choices"]):
            continue
        answer = ds[idx]["answer_choices"][pred_index]
        if answer == ds[idx]["correct_answer"]:
            correct_idx.append(idx)
            for item in data:
                processed_data.append(parse_image_data(item))
            
print(len(correct_idx)/total_count)
print(len(correct_idx))
print(total_count)
        
            

  8%|▊         | 81/1019 [00:32<07:24,  2.11it/s]

The question cannot be answered as there are no separate gray rectangle sofa and green threecushion sofa in the image


 11%|█         | 113/1019 [00:47<07:30,  2.01it/s]

The question cannot be answered from the provided image


 25%|██▌       | 258/1019 [01:32<02:09,  5.89it/s]

...

Neither A nor B can be definitively chosen as the objects are at the same depth


 91%|█████████ | 926/1019 [04:00<00:13,  6.90it/s]

The question cannot be answered as there is no doorway in the image


100%|██████████| 1019/1019 [04:20<00:00,  3.91it/s]


0.6437684003925417
656
1019


In [48]:
output_data = []

conv_type_counter = {
}

for item in processed_data:
    
    if item["conv_type"] not in conv_type_counter:
        conv_type_counter[item["conv_type"]] = 0
    conv_type_counter[item["conv_type"]] += 1

    entry = {
        "messages": [],
        "images": [],
    }

    for msg in item["messages"]:
        if msg["role"] == "user":
            for sub_item in msg["content"]:
                if sub_item["type"] == "image":
                    entry["images"].append(sub_item["image"])
                elif sub_item["type"] == "text":
                    entry["messages"].append({
                        "content": sub_item["text"],
                        "role": "user"
                    })
                else:
                    print("Unknown message type")
        elif msg["role"] == "system":
            if len(msg["content"]) != 1:
                print("ERROR")

            entry["messages"].append({
                "content": msg["content"][0]["text"],
                "role": "assistant"
            })
        else:
            print("Unknown message role")
    
    entry["messages"].append({
        "content": item["response"],
        "role": "assistant"
    })
        
    output_data.append(entry)

print(conv_type_counter)

{'get_objects_of_interest': 656, 'detection_refinement': 1366, 'get_reference_viewer': 656, 'convert_to_ego': 656, 'perspective_visual': 656, 'abstract_to_real': 656}


In [49]:
with open(SAT_TRAIN_GENERATED_FOLDER + "/train_data.json", "w") as f:
    json.dump(output_data, f, indent=2)

In [50]:
print(image_idx_counter)

3663


In [ ]:
SAT_DATASET_FOLDER = "/ocean/projects/cis250208p/shared/datasets/SAT"
DATASET_FOLDER = "/ocean/projects/cis250208p/shared/datasets"
# with open(os.path.join(SAT_DATASET_FOLDER, "data.json"), "w") as f:

merge_data = []

with open(os.path.join(SAT_DATASET_FOLDER, "train_generated/train_data.json"), "r") as f:
    merge_data.extend(json.load(f))

with open(os.path.join(SAT_DATASET_FOLDER, "train_generated_1/train_data.json"), "r") as f:
    merge_data.extend(json.load(f))

with open(os.path.join(DATASET_FOLDER, "data.json"), "w") as f:
    json.dump(merge_data, f, indent=2)